In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# ============================
# 1) Load Dataset
# ============================
df = pd.read_csv("train_u6lujuX_CVtuZ9i (1).csv")

# Drop Loan_ID (identifier)
df.drop(columns=["Loan_ID"], inplace=True)

# Create TotalIncome feature
df["TotalIncome"] = df["ApplicantIncome"] + df["CoapplicantIncome"]

# Convert Dependents "3+" → "3"
df["Dependents"] = df["Dependents"].replace("3+", "3")

# Map target variable: Y → 1, N → 0
df["Loan_Status"] = df["Loan_Status"].map({"Y": 1, "N": 0})

# ============================
# 2) Select Features & Target
# ============================
features = [
    "Gender", "Married", "Dependents", "Education", "Self_Employed",
    "Credit_History", "Property_Area", "LoanAmount",
    "Loan_Amount_Term", "TotalIncome"
]

X = df[features]
y = df["Loan_Status"]

# ============================
# 3) Define Numeric & Categorical Columns
# ============================
numeric_features = ["LoanAmount", "Loan_Amount_Term", "TotalIncome", "Credit_History"]
categorical_features = ["Gender", "Married", "Dependents", "Education", "Self_Employed", "Property_Area"]

# ============================
# 4) Preprocessing Pipelines
# ============================

# Numeric pipeline → impute (median) + scale
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical pipeline → impute (most frequent) + OneHot Encoding
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combined preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# ============================
# 5) Train/Test Split (70/30)
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

# ============================
# 6) Fit Preprocessing Only
# ============================
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

print("Preprocessed training shape:", X_train_prepared.shape)
print("Preprocessed testing shape:", X_test_prepared.shape)


Training set shape: (429, 10)
Testing set shape: (185, 10)
Preprocessed training shape: (429, 19)
Preprocessed testing shape: (185, 19)


In [3]:
# =============================
# 📌 Logistic Regression Model Training
# =============================

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# ---------------------------------------------------
# 1) Create a full pipeline:
#    - Preprocessing (that we built earlier)
#    - Logistic Regression model
# ---------------------------------------------------

log_reg_model = Pipeline(steps=[
    ("preprocessing", preprocessor),          # خطوات الـ preprocessing التي عملناها
    ("classifier", LogisticRegression())      # نموذج الانحدار اللوجستي
])

# ---------------------------------------------------
# 2) Train the model on training data
# ---------------------------------------------------
log_reg_model.fit(X_train, y_train)
# شرح: هنا يتم تدريب النموذج على بيانات التدريب بعد تطبيق كل خطوات الـ preprocessing عليها.

# ---------------------------------------------------
# 3) Evaluate the model on test data
# ---------------------------------------------------
log_reg_accuracy = log_reg_model.score(X_test, y_test)
# شرح: score يعطي دقة النموذج accuracy

print("Logistic Regression Accuracy:", log_reg_accuracy)


Logistic Regression Accuracy: 0.8540540540540541


In [5]:
# =====================================================
# 📌 Logistic Regression with Different Penalties (L1 & L2)
# =====================================================

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# -----------------------------------------------------
# 1) Logistic Regression with L2 regularization (default)
# -----------------------------------------------------

log_reg_L2 = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200))
])

# Train
log_reg_L2.fit(X_train, y_train)

# Accuracy
acc_L2 = log_reg_L2.score(X_test, y_test)
print("Logistic Regression (L2) Accuracy:", acc_L2)


# -----------------------------------------------------
# 2) Logistic Regression with L1 regularization
# -----------------------------------------------------
# ملاحظة: penalty='l1' يحتاج solver اسمه 'liblinear' أو 'saga'

log_reg_L1 = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(penalty='l1', solver='liblinear', max_iter=200))
])

# Train
log_reg_L1.fit(X_train, y_train)

# Accuracy
acc_L1 = log_reg_L1.score(X_test, y_test)
print("Logistic Regression (L1) Accuracy:", acc_L1)


Logistic Regression (L2) Accuracy: 0.8540540540540541
Logistic Regression (L1) Accuracy: 0.8486486486486486


In [7]:
# ===========================================
# 📌 Evaluate Logistic Regression Models (L1 & L2)
# ===========================================

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ------------------------------------------------
# 1) Predict using both models
# ------------------------------------------------

y_pred_L2 = log_reg_L2.predict(X_test)
y_pred_L1 = log_reg_L1.predict(X_test)

# ------------------------------------------------
# 2) Define a function to print evaluation metrics
# ------------------------------------------------

def evaluate_model(y_true, y_pred, model_name):
    print(f"\n===== {model_name} =====")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, pos_label="Y"))
    print("Recall   :", recall_score(y_true, y_pred, pos_label="Y"))
    print("F1 Score :", f1_score(y_true, y_pred, pos_label="Y"))

# ------------------------------------------------
# 3) Evaluate both models
# ------------------------------------------------

evaluate_model(y_test, y_pred_L2, "Logistic Regression (L2 Penalty)")
evaluate_model(y_test, y_pred_L1, "Logistic Regression (L1 Penalty)")



===== Logistic Regression (L2 Penalty) =====
Accuracy : 0.8540540540540541


ValueError: pos_label=Y is not a valid label. It should be one of [0, 1]

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict
y_pred_L2 = log_reg_L2.predict(X_test)
y_pred_L1 = log_reg_L1.predict(X_test)

# Function to evaluate model
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n===== {model_name} =====")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, pos_label=1))
    print("Recall   :", recall_score(y_true, y_pred, pos_label=1))
    print("F1 Score :", f1_score(y_true, y_pred, pos_label=1))

# Evaluate both models
evaluate_model(y_test, y_pred_L2, "Logistic Regression (L2 Penalty)")
evaluate_model(y_test, y_pred_L1, "Logistic Regression (L1 Penalty)")



===== Logistic Regression (L2 Penalty) =====
Accuracy : 0.8540540540540541
Precision: 0.8333333333333334
Recall   : 0.984251968503937
F1 Score : 0.9025270758122743

===== Logistic Regression (L1 Penalty) =====
Accuracy : 0.8486486486486486
Precision: 0.8278145695364238
Recall   : 0.984251968503937
F1 Score : 0.8992805755395683


In [11]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# --------------------------------------------
# 1) Define pipeline (preprocessing + classifier)
# --------------------------------------------
pipe = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(solver='liblinear', max_iter=500, random_state=42))
])

# --------------------------------------------
# 2) Define hyperparameter grid
# --------------------------------------------
param_grid = {
    "classifier__penalty": ["l1", "l2"],   # نوع الـ Regularization
    "classifier__C": [0.01, 0.1, 1, 10, 100]  # قوة الـ regularization
}

# --------------------------------------------
# 3) Setup GridSearchCV
# --------------------------------------------
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',   # يمكن تغييره إلى 'roc_auc' إذا أردت
    n_jobs=-1,
    verbose=1
)

# --------------------------------------------
# 4) Train model (fit GridSearchCV)
# --------------------------------------------
grid.fit(X_train, y_train)

# --------------------------------------------
# 5) Best parameters and score
# --------------------------------------------
print("Best parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

# --------------------------------------------
# 6) Evaluate on test set
# --------------------------------------------
y_pred = grid.predict(X_test)

print("\nTest set evaluation:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, pos_label=1))
print("Recall   :", recall_score(y_test, y_pred, pos_label=1))
print("F1 Score :", f1_score(y_test, y_pred, pos_label=1))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'classifier__C': 0.01, 'classifier__penalty': 'l2'}
Best CV Accuracy: 0.7924760601915185

Test set evaluation:
Accuracy : 0.8486486486486486
Precision: 0.8278145695364238
Recall   : 0.984251968503937
F1 Score : 0.8992805755395683

Confusion Matrix:
[[ 32  26]
 [  2 125]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9412    0.5517    0.6957        58
           1     0.8278    0.9843    0.8993       127

    accuracy                         0.8486       185
   macro avg     0.8845    0.7680    0.7975       185
weighted avg     0.8634    0.8486    0.8354       185



In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ---------------------------------------------------
# 1) Define pipeline (preprocessing + LogisticRegression)
# ---------------------------------------------------
pipe = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# ---------------------------------------------------
# 2) Define hyperparameter grid
# ---------------------------------------------------
param_grid = {
    "classifier__penalty": ["l1", "l2", "elasticnet", "none"],    # أنواع regularization
    "classifier__C": [0.01, 0.1, 1, 10, 100],                    # قوة regularization
    "classifier__solver": ["saga"],                               # 'saga' يدعم elasticnet و l1/l2
    "classifier__l1_ratio": [0, 0.5, 0.7, 1]                     # فقط لـ elasticnet
}

# ---------------------------------------------------
# 3) Setup GridSearchCV
# ---------------------------------------------------
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='accuracy',      # يمكن تغييره إلى 'roc_auc' حسب الرغبة
    n_jobs=-1,
    verbose=1
)

# ---------------------------------------------------
# 4) Train model (fit GridSearchCV)
# ---------------------------------------------------
grid.fit(X_train, y_train)

# ---------------------------------------------------
# 5) Best parameters & best CV score
# ---------------------------------------------------
print("Best parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

# ---------------------------------------------------
# 6) Evaluate on test set
# ---------------------------------------------------
y_pred = grid.predict(X_test)

print("\nTest set evaluation:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, pos_label=1))
print("Recall   :", recall_score(y_test, y_pred, pos_label=1))
print("F1 Score :", f1_score(y_test, y_pred, pos_label=1))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'classifier__C': 0.01, 'classifier__l1_ratio': 0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Best CV Accuracy: 0.7924760601915185

Test set evaluation:
Accuracy : 0.8486486486486486
Precision: 0.8278145695364238
Recall   : 0.984251968503937
F1 Score : 0.8992805755395683

Confusion Matrix:
[[ 32  26]
 [  2 125]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9412    0.5517    0.6957        58
           1     0.8278    0.9843    0.8993       127

    accuracy                         0.8486       185
   macro avg     0.8845    0.7680    0.7975       185
weighted avg     0.8634    0.8486    0.8354       185



C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\pipeline.py", l

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ---------------------------------------------------
# 1) Pipeline (Preprocessing + Logistic Regression)
# ---------------------------------------------------
pipe = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# ---------------------------------------------------
# 2) Hyperparameter grid
# ---------------------------------------------------
param_grid = {
    "classifier__penalty": ["l1", "l2", "elasticnet", "none"],  # أنواع regularization
    "classifier__C": [0.01, 0.1, 1, 10, 100],                  # قوة regularization
    "classifier__solver": ["saga"],                             # saga يدعم elasticnet و l1/l2
    "classifier__l1_ratio": [0, 0.5, 0.7, 1]                   # فقط لـ elasticnet
}

# ---------------------------------------------------
# 3) GridSearchCV with stratified folds for balanced splits
# ---------------------------------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# ---------------------------------------------------
# 4) Fit GridSearchCV
# ---------------------------------------------------
grid.fit(X_train, y_train)

# ---------------------------------------------------
# 5) Best parameters and CV score
# ---------------------------------------------------
print("===== Best Parameters =====")
print(grid.best_params_)
print("Best Cross-Validation Accuracy:", grid.best_score_)

# ---------------------------------------------------
# 6) Evaluate best model on test set
# ---------------------------------------------------
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print("\n===== Test Set Evaluation =====")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, pos_label=1))
print("Recall   :", recall_score(y_test, y_pred, pos_label=1))
print("F1 Score :", f1_score(y_test, y_pred, pos_label=1))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
===== Best Parameters =====
{'classifier__C': 0.01, 'classifier__l1_ratio': 0, 'classifier__penalty': 'l2', 'classifier__solver': 'saga'}
Best Cross-Validation Accuracy: 0.7924760601915185

===== Test Set Evaluation =====
Accuracy : 0.8486486486486486
Precision: 0.8278145695364238
Recall   : 0.984251968503937
F1 Score : 0.8992805755395683

Confusion Matrix:
[[ 32  26]
 [  2 125]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9412    0.5517    0.6957        58
           1     0.8278    0.9843    0.8993       127

    accuracy                         0.8486       185
   macro avg     0.8845    0.7680    0.7975       185
weighted avg     0.8634    0.8486    0.8354       185



C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LEGION\.anaconda\anacoda 4\Lib\site-packages\sklearn\pipeline.py", 

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# ==============================
# 1) Define models to compare
# ==============================
models = {
    "Logistic Regression L2": log_reg_L2,
    "Logistic Regression L1": log_reg_L1,
    "Logistic Regression ElasticNet": log_reg_EN,
    "Decision Tree": decision_tree_model
}

# ==============================
# 2) Calculate metrics for each model
# ==============================
metrics_summary = []

for name, model in models.items():
    y_pred = model.predict(X_test)
    metrics_summary.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, pos_label=1),
        "Recall": recall_score(y_test, y_pred, pos_label=1),
        "F1 Score": f1_score(y_test, y_pred, pos_label=1)
    })

metrics_df = pd.DataFrame(metrics_summary)
print(metrics_df)

# ==============================
# 3) Bar plot of metrics
# ==============================
metrics_melted = metrics_df.melt(id_vars="Model", var_name="Metric", value_name="Score")

plt.figure(figsize=(10,6))
sns.barplot(data=metrics_melted, x="Model", y="Score", hue="Metric")
plt.title("Comparison of Performance Metrics")
plt.ylim(0,1)
plt.legend(loc="lower right")
plt.xticks(rotation=20)
plt.show()

# ==============================
# 4) Confusion Matrix for each model
# ==============================
for name, model in models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {name}")
    plt.show()


NameError: name 'log_reg_EN' is not defined